# Monitor Script

This is an example script monitoring values form the Enviro board and the external temperature sensor. I tried to organize the code in a way that makes it easy learn and to use parts of it for your scripts.

In [1]:
# Needed to communicate with the BME280 sensor (temperature, humidity, pressure)

from bme280 import BME280
bme280 = BME280()

In [2]:
# Google sheet magic

import gspread
from oauth2client.service_account import ServiceAccountCredentials

sheet_name = "PiLog"   # Make sure this matches your Google sheet name

# use creds to create a client to interact with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

In [3]:
# This is needed if you want to use the light sensor

from ltr559 import LTR559
ltr559 = LTR559()

In [4]:
# Added code to read the CCS811 sensor values

import busio
import board
import adafruit_ccs811

i2c = board.I2C() 
ccs = adafruit_ccs811.CCS811(i2c)

while (not ccs.data_ready):
    pass

print ("CCS811 Sensor ready.")

CCS811 Sensor ready.


In [5]:
# This is what we need to read the external temperature sensor

import smbus

# Simple helper routine to convert the reading to degrees C
def readTemp():
    # By default the address of LM75A is set to 0x48
    address = 0x48

    # Read I2C data and calculate temperature
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [6]:
# This is what we need to communicate with the Adafruit Cloud

from Adafruit_IO import Client, Feed, RequestError

ADAFRUIT_IO_USERNAME = "tproffen"
ADAFRUIT_IO_KEY = "ed44b8031ec34d1fb2eda6f4d8a0c832"

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

In [7]:
# Routines to write to the LCD screen
import ST7735
from PIL import Image, ImageDraw, ImageFont
from fonts.ttf import RobotoMedium as UserFont

disp = ST7735.ST7735(port=0,cs=1,dc=9,backlight=12,rotation=270,spi_speed_hz=10000000)
disp.begin()

back_color = (0, 60, 20)
text_color = (255, 255, 255)
head_color = (255, 255,   0)
warn_color = (255,  50,  50)
font_size  = 14
text_margin= 5

img = Image.new('RGBA', (disp.width, disp.height))
draw = ImageDraw.Draw(img)
font = ImageFont.truetype(UserFont, font_size)
font_big = ImageFont.truetype(UserFont, 2*font_size)

size_x, size_y = draw.textsize('text', font)

# We pass the name and the value so we can 'cycle' through the sensors
def updateLCD(label, value, unit): 
    if (value < 1000):
        vstr="{0:.1f} {1}".format(value, unit)
    else:
        vstr="{0:.0f} {1}".format(value, unit)
        
    draw.rectangle((0, 0, disp.width, disp.height), back_color)
    draw.text((text_margin, text_margin), "ORCSPICamp Station", font=font, fill=head_color)
    draw.text((text_margin, text_margin+2.0*size_y), label, font=font, fill=text_color)
    draw.text((text_margin, text_margin+3.5*size_y), vstr, font=font_big, fill=text_color)
    disp.display(img)

In [8]:
# Some general settings
import time
from datetime import datetime
from IPython.display import clear_output, display, update_display

# Get feeds using our routine
tempFeed  = aio.feeds("temperature")
temp2Feed = aio.feeds("tempexternal")
humidFeed = aio.feeds("humidity")
pressFeed = aio.feeds("pressure")
lightFeed = aio.feeds("brightness")
co2Feed   = aio.feeds("co2")
tvocFeed  = aio.feeds("tvoc")

# Set metadata associated with our measurement station. Update for your location
metadata = {'lat': 36.010357, 'lon': -84.269646, 'ele': 850, 'created_at': None}

# Time between readings in seconds
delay = 60

# Other settings for timing and LCD cycling

start_time   = time.time()
current_time = start_time
update_data  = 60.0
update_lcd   = 3.0
next_lcd     = 't'


## Main loop

In [ ]:
running = True

while running:
    try:
        # Only do the data update if elapsed time is multiple of upodate_data
        if ((int (current_time - start_time) % update_data) == 0):
            p  = bme280.get_pressure()
            t  = bme280.get_temperature()
            h  = bme280.get_humidity()
            l  = ltr559.get_lux()
            c  = ccs.eco2
            v  = ccs.tvoc
            t2 = readTemp() 

            clear_output(wait=True)

            out="T1: {0:.1f} C - T2: {1:.1f} C - Humidity: {2:.1f} % - Pressure: {3:.0f} hPa".format(t,t2,h,p)
            print(out)
            out="CO2: {0:.1f} ppm - TVOC: {1:.1f} ppb".format(c,v)
            print(out)

            # Sending data to Adafruit cloud
            aio.send_data(tempFeed.key,  t, metadata)
            aio.send_data(temp2Feed.key, t2, metadata)
            aio.send_data(humidFeed.key, h, metadata)
            aio.send_data(pressFeed.key, p, metadata)
            aio.send_data(lightFeed.key, l, metadata)
            aio.send_data(co2Feed.key, c, metadata)
            aio.send_data(tvocFeed.key, v, metadata)

            # Sending data to Google Sheet
            now=datetime.now()
            now_time = now.strftime("%H:%M:%S")
            now_day = now.strftime("%m/%d/%Y")
            row=[now_day, now_time, t, t2, h, p, l, c, v]
            sheet.append_row(row)
        
        # We update the LCD every cycle
        if (next_lcd == 't'):
            updateLCD('Temperature', t, 'C')
            next_lcd = 't2'
        elif (next_lcd == 't2'):
            updateLCD('Temperature (ext)', t2, 'C')
            next_lcd = 'h'
        elif (next_lcd == 'h'):
            updateLCD('Humidity', h, '%')
            next_lcd = 'p'
        elif (next_lcd == 'p'):
            updateLCD('Pressure', p, 'hPa')
            next_lcd = 'c'
        elif (next_lcd == 'c'):
            updateLCD('CO2', c, 'ppm')
            next_lcd = 'v'
        elif (next_lcd == 'v'):
            updateLCD('TVOC', v, 'ppb')
            next_lcd = 't'

        current_time = time.time()
        time.sleep(update_lcd)
    except IOError:
        pass
    except KeyboardInterrupt:
        print ("Loop terminated by user.")
        running = False


T1: 22.1 C - T2: 23.8 C - Humidity: 86.5 % - Pressure: 706 hPa
CO2: 402.0 ppm - TVOC: 0.0 ppb
